# Referencing Partial State Update Blocks labels to substeps in cadCAD

*Vitor Marthendal Nunes*

---

This notebook shows how to use label metadata on PSUBs in order to do post-processing on the simulation.  

## Dependences

In [1]:
%%capture
!pip install cadcad 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor

## Definitions

### Initial conditions and parameters

In [3]:
initial_conditions = {
    'prey_population': 100,
    'predator_population': 15
    }

params = {
    "prey_birth_rate": [1.0],
    "predator_birth_rate": [0.01],
    "predator_death_const": [1.0],
    "prey_death_const": [0.03],
    "dt": [0.1] # Precision of the simulation. Lower is more accurate / slower
}

simulation_parameters = {
    'N': 1,
    'T': range(30),
    'M': params
}

### Policies

In [4]:
def p_predator_births(params, step, sL, s):
  dt = params['dt']
  predator_population = s['predator_population']
  prey_population = s['prey_population']
  birth_fraction = params['predator_birth_rate'] + np.random.random() * 0.0002
  births =  birth_fraction * prey_population * predator_population * dt
  return {'add_to_predator_population': births}


def p_prey_births(params, step, sL, s):
  dt = params['dt']
  population = s['prey_population']
  birth_fraction = params['prey_birth_rate'] + np.random.random() * 0.1
  births =  birth_fraction * population * dt
  return {'add_to_prey_population': births}


def p_predator_deaths(params, step, sL, s):
  dt = params['dt']
  population = s['predator_population']
  death_rate = params['predator_death_const'] + np.random.random() * 0.005
  deaths = death_rate * population * dt
  return {'add_to_predator_population': -1.0 * deaths}


def p_prey_deaths(params, step, sL, s):
  dt = params['dt']
  death_rate = params['prey_death_const'] + np.random.random() * 0.1
  prey_population = s['prey_population']
  predator_population = s['predator_population']
  deaths = death_rate * prey_population * predator_population * dt
  return {'add_to_prey_population': -1.0 * deaths}

### State update functions

In [5]:
def s_prey_population(params, step, sL, s, _input):
    y = 'prey_population'
    x = s['prey_population'] + _input['add_to_prey_population']
    return (y, x)


def s_predator_population(params, step, sL, s, _input):
    y = 'predator_population'
    x = s['predator_population'] + _input['add_to_predator_population']
    return (y, x)

### State update blocks

In [6]:
partial_state_update_blocks = [
    { 
        'label': 'Predator dynamics',
        'policies': {
            'predator_births': p_predator_births,
            'predator_deaths': p_predator_deaths
        },
        'variables': {
            'predator_population': s_predator_population            
        }
    },
    {
        'label': 'Prey dynamics',
        'policies': {
            'prey_births': p_prey_births,
            'prey_deaths': p_prey_deaths
        },
        'variables': {
            'prey_population': s_prey_population
        }
    }
]

### Configuration and Execution

In [7]:
sim_config = config_sim(simulation_parameters)

exp = Experiment()
exp.append_configs(sim_configs=sim_config, 
                   initial_state=initial_conditions,
                   partial_state_update_blocks=partial_state_update_blocks)


from cadCAD import configs
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.local_mode)
executor = Executor(exec_context=exec_context, configs=configs) 
(records, tensor_field, _) = executor.execute() 


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (30, 5, 1, 2)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.02s


### Results

In [8]:
df = pd.DataFrame(records)
df

,prey_population,predator_population,simulation,subset,run,substep,timestep
0,100.000000,15.000000,0,0,1,0,0
1,100.000000,15.016254,0,0,1,1,1
2,92.636912,15.016254,0,0,1,2,1
3,92.636912,14.920401,0,0,1,1,2
4,88.061100,14.920401,0,0,1,2,2
...,...,...,...,...,...,...,...
56,111.954585,9.728904,0,0,1,2,28
57,111.954585,9.853190,0,0,1,1,29
58,115.206591,9.853190,0,0,1,2,29
59,115.206591,10.009231,0,0,1,1,30


In [9]:
psubs = partial_state_update_blocks
psub_map = {order+1: psub['label'] for (order, psub) in enumerate(psubs)}

In [10]:
df['psubs'] = df.substep.map(psub_map)
df

,prey_population,predator_population,simulation,subset,run,substep,timestep,psubs
0,100.000000,15.000000,0,0,1,0,0,NaN
1,100.000000,15.016254,0,0,1,1,1,Predator dynamics
2,92.636912,15.016254,0,0,1,2,1,Prey dynamics
3,92.636912,14.920401,0,0,1,1,2,Predator dynamics
4,88.061100,14.920401,0,0,1,2,2,Prey dynamics
...,...,...,...,...,...,...,...,...
56,111.954585,9.728904,0,0,1,2,28,Prey dynamics
57,111.954585,9.853190,0,0,1,1,29,Predator dynamics
58,115.206591,9.853190,0,0,1,2,29,Prey dynamics
59,115.206591,10.009231,0,0,1,1,30,Predator dynamics


In [11]:
df.query("psubs=='Predator dynamics'")

,prey_population,predator_population,simulation,subset,run,substep,timestep,psubs
1,100.000000,15.016254,0,0,1,1,1,Predator dynamics
3,92.636912,14.920401,0,0,1,1,2,Predator dynamics
5,88.061100,14.759394,0,0,1,1,3,Predator dynamics
7,82.673564,14.512476,0,0,1,1,4,Predator dynamics
9,75.466382,14.154294,0,0,1,1,5,Predator dynamics
11,79.432579,13.863539,0,0,1,1,6,Predator dynamics
13,82.441356,13.637235,0,0,1,1,7,Predator dynamics
15,84.886092,13.431815,0,0,1,1,8,Predator dynamics
17,80.597110,13.184202,0,0,1,1,9,Predator dynamics
19,78.827053,12.920694,0,0,1,1,10,Predator dynamics


In [12]:
df.query("psub=='Prey dynamics'")

UndefinedVariableError: name 'psub' is not defined